In [29]:
import polars as pl

from pathlib import Path
import os
import json

In [2]:
DATA_DIR = Path.cwd().parent / "data" / "avito_ml_cup"

# Просмотр данных

**clickstream.pq**
- *cookie* - id пользователя [int64]
- *item* - id объявления [int64]
- *event* - id тип события [int64]. Например “клик”, “показать телефон”, “написать в мессенджер”
- *event_date* - время, когда пользователь провзаимодействовал с айтемом datetime[ns]
- *surface*  - экран, с которого было взаимодействие. Например пользователь может взаимодействовать с айтемом на поисковой выдаче, во вкладке “Избранное” [int64]
- *platform* -  тип устройства, с которого пользователь совершил действие. Например - android, ios, desktop, браузер в телефоне [int64]
- *node* - id группы товара. Сущность , которую необходимо предсказать [int64]

In [3]:
df_clickstream = pl.read_parquet(f'{DATA_DIR}/clickstream.pq', n_rows=1_000_000)

print(df_clickstream.shape)
df_clickstream.head()

(1000000, 7)


cookie,item,event,event_date,platform,surface,node
i64,i64,i64,datetime[ns],i64,i64,u32
0,19915558,17,2025-02-05 02:30:59,3,2,115659
0,2680232,17,2025-01-24 21:16:57,3,2,115829
1,4247649,17,2025-01-29 23:00:58,2,2,7
1,4247649,17,2025-02-17 14:55:17,2,2,7
1,2171135,17,2025-01-17 19:23:29,2,2,214458


In [68]:
df_clickstream["event"].n_unique()
df_clickstream["event"].value_counts().sort(by="count", descending=True).head(10)

event,count
i64,u32
17,888775
11,62748
12,12209
10,11896
15,4851
3,4246
5,4178
19,2785
8,2291


In [12]:
df_clickstream["surface"].value_counts().sort(by="count", descending=True).head(10)

surface,count
i64,u32
2,481762
11,298096
3,58409
5,58024
15,40741
8,28916
17,16016
14,13520
4,1241


In [16]:
df_clickstream["platform"].value_counts().sort(by="count", descending=True)

platform,count
i64,u32
2,587621
3,350551
0,32648
5,28955
1,141
4,84


In [17]:
df_clickstream["node"].n_unique()

73227

**cat_features.pq**
- *item* -  id объявления [int64]
- *location* - id локации айтема.  [int64]
- *category* - id категории товара. [int64]
- *clean_params* - параметры объявления. В примере [{"attr":859,"value":653982} …]  attr - это id атрибута, value - это id значения атрибута [str]
- *node* - id группы товара. Сущность , которую необходимо предсказать [int64]

In [18]:
df_cat_features = pl.read_parquet(f'{DATA_DIR}/cat_features.pq', n_rows=1_000_000)

df_cat_features.head()

item,location,category,clean_params,node
i64,i64,i64,str,u32
9,8385,57,"""[{""attr"":1157,""value"":664427},…",194747
17,2707,35,"""[{""attr"":2140,""value"":501466},…",352905
144,8383,8,"""[{""attr"":802,""value"":35791},{""…",17188
202,5397,57,"""[{""attr"":1157,""value"":490527},…",194766
236,2105,64,"""[{""attr"":112,""value"":420797},{…",153951


In [49]:
json_feature = json.loads(df_cat_features["clean_params"].sample(1)[0])

len(json_feature)

16

In [ ]:
df_cat_features["location"].n_unique()

3688

In [54]:
df_cat_features["category"].value_counts().sort(by="count", descending=True).head(10)

category,count
i64,u32
35,162936
51,103040
19,56689
24,50747
40,49328
7,43318
37,41374
57,40553
46,40488


In [55]:
df_cat_features["node"].n_unique()

101173

**text_features.pq**
- *item*  - id объявления [int64]
- *title_projection* - вектор тайтла объявления array[int8, 64]. Чем выше скалярное произведение между 2-мя векторами, тем более похожи заголовки объявлений

In [56]:
df_text_features = pl.read_parquet(f'{DATA_DIR}/text_features.pq', n_rows=100_000)

df_text_features.head()

item,title_projection
i64,"array[i8, 64]"
9,"[-128, 90, … -36]"
17,"[-128, 127, … -3]"
144,"[-128, 97, … 18]"
202,"[-128, 127, … 15]"
236,"[-128, 29, … -128]"


**events.pq**
- *event* - id типы события [int64]
- *is_contact* - 0 или 1. Является ли событие контактным [int64]

In [71]:
df_events = pl.read_parquet(f'{DATA_DIR}/events.pq')

df_events.head()

event,is_contact
i64,i64
0,1
11,0
4,1
16,0
15,1


In [78]:
df_events.sort("is_contact", descending=True)

event,is_contact
i64,i64
0,1
4,1
15,1
18,1
5,1
…,…
16,0
3,0
12,0
